# Create Spotify playlist from list

丁曉雯

In [3]:
import requests
import json
import re
import pandas as pd

def prettify(json_file):
    """output pretty json
    """
    parsed = json.loads(json_file)
    return json.dumps(parsed, indent=4, sort_keys=True)

def append_song(text):
    """append track id to a list"""
    track = re.findall(r'"spotify:track:(.*?)"', text)[0]
    if track is not None:
        song_list.append(track)
    return song_list

def search_song(term):
    """search for songs on spotify and add to list
    """
    url = "https://api.spotify.com/v1/search?q="+term+"&type=track"
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.get(url, headers=headers)
    parsed_json = json.loads(r.text)
    if parsed_json['tracks']['total'] != 0:
        append_song(r.text)
    else:
        # song unavailable
        return False
    return True

def add_song(term):
    """add song to spotify playlist
    """
    url = "https://api.spotify.com/v1/playlists/"+playlist_id+"/tracks?uris=spotify%3Atrack%3A"+term
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.post(url, headers=headers)
#     print(r.text)
    return

In [130]:
# https://developer.spotify.com/console/post-playlist-tracks/?playlist_id=&position=&uris=
song_list = []
na_list = []
playlist_id = '6srqweC5oZeihgMnbbnUNE'
link = "https://zh.wikipedia.org/wiki/%E7%AC%AC%E5%8D%81%E4%B8%83%E5%B1%86%E5%8D%81%E5%A4%A7%E4%B8%AD%E6%96%87%E9%87%91%E6%9B%B2%E5%BE%97%E7%8D%8E%E5%90%8D%E5%96%AE"
token = "BQA85c9Ba0u9H1nk-HalaPc9nGKOjAblafLTFBQlWu_dPl34Q_HUDt8uCVTH7-hiXuMnDe2POienBZpVd-X1W3J3jqlHqc7xR96WvQ-ekOTDRAlI5jnrQcNic2iWivNZp0rCTU-mbSYV7yFR1StG-BKrOAx59NXttr09_2_JdWRa24ybiFJrQAHnkUerwx-8oUnTgA"

text = """    《我們的主題曲》（主唱：鄭秀文 作曲：吳國敬 填詞：黃偉文）
    《愛的呼喚》（主唱：郭富城 作曲：譚國政 填詞：小美）
    《不老的傳說》（主唱：張學友 作曲：Dick Lee 填詞：古倩敏）
    《明知故犯》（主唱：許美靜 作曲：陳佳明 填詞：林夕）
    《星夢情真》（主唱：陳慧琳 作曲：雷頌德 填詞：黃偉文）
    《只要為我愛一天》（主唱：黎明 作曲：雷頌德 填詞：林夕）
    《歡樂今宵》（主唱：古巨基 作曲：黃丹儀 填詞：黃偉文）
    《中國人》（主唱：劉德華 作曲：陳耀川 填詞：李安修）
    《你快樂所以我快樂》（主唱：王菲 作曲：張亞東 填詞：林夕）
    《愛是永恆》（主唱：張學友 作曲：Dick Lee 填詞：林振強）"""

## Text

In [131]:
songs_to_search = text.splitlines()

for i in range(len(songs_to_search)):
    songs_to_search[i] = songs_to_search[i].split('––')[0]
    songs_to_search[i] = songs_to_search[i].split('——')[0]
    songs_to_search[i] = songs_to_search[i].split('－')[0]
    songs_to_search[i] = songs_to_search[i].split('（')[0]
    if '《' in songs_to_search[i]:
        songs_to_search[i] = re.findall(r'《(.*?)》', songs_to_search[i])[0]

# songs_to_search

## HTML

In [ ]:
website = pd.read_html(link)

In [107]:
all_songs = website[1]
all_songs

十大中文金曲得獎名單（按頒獎次序排序）                                                  
                   編號       歌曲                 作曲    作詞   主唱         專輯
0                  64     餓狼傳說                劉諾生   潘偉源  张学友     《餓狼傳說》
1                  36  那有一天不想你                林慕德   向雪懷   黎明     《天地情緣》
2                  15    女人的弱點     CHAGE and ASKA   潘源良  葉蒨文    《女人的弱點》
3                  69    心酸的情歌                巫啟賢   劉德華  巫啟賢    《心酸的情歌》
4                  68    繾綣星光下  Jl DabadieLunghin   周禮茂  關淑怡   《My Way》
5                  79   忘情水（國）                陳耀川   李安修  劉德華      《忘情水》
6                  88     鐵幕誘惑                譚國政  小美林夕  郭富城     《鐵幕誘惑》
7                  86      誰人知                張影坤    林夕  劉德華    《5時30分》
8                   1       冷戰          Amos Tori    林夕  王靖雯  《十萬個為什麼？》
9                  67     來來回回                杉田裕   周禮茂  張學友     《餓狼傳說》

In [111]:
all_songs.columns

MultiIndex([('十大中文金曲得獎名單（按頒獎次序排序）',     '編號'),
            ('十大中文金曲得獎名單（按頒獎次序排序）',  'title'),
            ('十大中文金曲得獎名單（按頒獎次序排序）',     '作曲'),
            ('十大中文金曲得獎名單（按頒獎次序排序）',     '作詞'),
            ('十大中文金曲得獎名單（按頒獎次序排序）', 'artist'),
            ('十大中文金曲得獎名單（按頒獎次序排序）',     '專輯')],
           )

In [112]:
all_songs.rename(columns={"歌曲": "title", "主唱": "artist"}, inplace=True)
# all_songs['title'] = all_songs['title'].apply(lambda x: re.findall(r'《(.*?)》', x)[0])
songs_to_search = all_songs[('十大中文金曲得獎名單（按頒獎次序排序）',  'title')]
# all_songs[['title', 'artist']]

## Create playlist

In [132]:
for i in songs_to_search:
    if not search_song(i):
        print(" unavailable          ", end="")
    print(i)
#     print(all_songs[all_songs['title']==i][['title', 'artist']].to_string(header=False))
    
# create playlist and add
for i in song_list:
    add_song(i)

我們的主題曲
愛的呼喚
不老的傳說
明知故犯
星夢情真
只要為我愛一天
歡樂今宵
中國人
你快樂所以我快樂
愛是永恆
